In [1]:
import numpy as np
from sympy import Matrix
import sympy as sp

Hill Cipher implementation:
<ul>
    <li>$\text{keyToMatrix()}$: convert a 4-letter key to 2x2 sympy.Matrix() object</li>
    <li>$\text{encodeSequence()}$: encode a string (+append 'a' if string length is odd)</li>
    <li>$\text{encodeDoubleSymbolSeq()}$: encode a string of length 2</li>
    <li>$\text{decodeDoubleSymbolSeq()}$: obvious</li>
    <li>$\text{decodeSequence()}$: decode a string of even length</li>
</ul>

In [2]:
class HillCipher:
    
    def __init__(self,key="chlf"):
        self.alphabet = [chr(x) for x in range(ord('a'), ord('z')+1)]
        self.alpha = dict((self.alphabet[i],i) for i in range(len(self.alphabet)))
        self.key = self.keyToMatrix(key)
        self.key_inv = self.key.inv_mod(len(self.alphabet))
    
    def keyToMatrix(self,key):
        assert len(key)==4
        nums = tuple(self.alpha[x] for x in key)
        return Matrix([[nums[0],nums[1]],
                      [nums[2],nums[3]]])
    
    def encodeSequence(self,string):
        if len(string)%2!=0:
            string+='a'
        return "".join(self.encodeDoubleSymbolSeq(string[i:i+2]) for i in range(0,len(string),2))
    
    def encodeDoubleSymbolSeq(self,seq:str):
        assert len(seq)==2
        nums = tuple(self.alpha[x] for x in seq)
        s = Matrix([[nums[0]],[nums[1]]])
        enc = self.key.multiply(s)%len(self.alphabet)
        return "".join((self.alphabet[i] for i in enc)).upper()
    
    def decodeDoubleSymbolSeq(self,seq:str):
        assert len(seq)==2
        nums = tuple(self.alpha[x] for x in seq)
        s = Matrix([nums[0],nums[1]])
        dec = self.key_inv.multiply(s)%len(self.alphabet)
        return "".join((self.alphabet[i] for i in dec)).upper()
    
    def decodeSequence(self,string):
        string=string.lower()
        assert len(string)%2==0
        return "".join(self.decodeDoubleSymbolSeq(string[i:i+2]) for i in range(0,len(string),2))


HillKeyDeduction:
<ul>
    <li>$\text{forceDeduceKey()} - \text{complicity}: O(n^4), n = \text{length(alphabet)}$</li> 
    <li>$\text{deduceKeyWithGaussMethod() - }\text{complicity}: O(n), \text{n = len(ciphertext)}$ - use the Gauss method instead (sympy.linsolve)</li> 
</ul>

In [3]:
class HillKeyDeduction:
    
    def __init__(self, string, encoded_string):
        self.alphabet = HillCipher().alphabet
        self.alpha = HillCipher().alpha
        if len(string)%2!=0:
            string+='a'
        self.string = string
        self.encoded_string = encoded_string
    
    def forceDeduceKey(self):
        def comparePlainTextAndCiphertext(*args):
            key = "".join(self.alphabet[x] for x in args)
            h = None
            try:
                h = HillCipher(key)
            except:
                return None
            return key if h.encodeSequence(self.string).lower()==self.encoded_string else None
            
        def _forceDeduceKey(i,j):
            nonlocal comparePlainTextAndCiphertext
            for k in range(len(self.alphabet)):
                for l in range(len(self.alphabet)):
                    key = comparePlainTextAndCiphertext(i,j,k,l) 
                    if key is not None:
                        return key 
            return None
        for i in range(len(self.alphabet)):
            for j in range(len(self.alphabet)):
                key = _forceDeduceKey(i,j)
                if key is not None:
                    return key 
        raise Exception("Unable to find the key")
    
    def deduceKeyWithGaussMethod(self):
        """
        For each double sequence, count = n, (x1,x2) - plain; (y1,y2) - decoded:
        [a, b] * [x1] = [y1, y2]
        [c, d]   [x2]   
        
        E0: a*x10 + b*x20 = y10
        E1: a*x11 + b*x21 = y11
        ...
        E_N: a*x1N + b*x2N = y1N
        
        M0: c*x10 + d*x20 = y20
        M1: c*x11 + d*x21 = y21
        ...
        M_N: c*x1 + d*x2 = y21
        """
        
        if (len(self.string)<4):
            raise Exception("impossible to deduce with the Gauss method")
        
        for i in range(0,len(self.string),4):
        
            txt = self.string[i:i+4]
            enc = self.encoded_string[i:i+4]

            x10 = self.alpha[txt[0]]
            x20 = self.alpha[txt[1]]
            x11 = self.alpha[txt[2]]
            x21 = self.alpha[txt[3]]

            y10 = self.alpha[enc[0]]
            y20 = self.alpha[enc[1]]
            y11 = self.alpha[enc[2]]
            y21 = self.alpha[enc[3]]

            a,b,c,d = sp.symbols('a,b,c,d')

            solution_ab = sp.linsolve([
                a*x10 + b*x20 - y10,
                a*x11 + b*x21 - y11
            ], (a,b))

            solution_cd = sp.linsolve([
                c*x10 + d*x20 - y20,
                c*x11 + d*x21 - y21,
            ], (c,d))

            n = 26
            a0 = solution_ab.args[0][0].as_numer_denom()[0]
            b0 = solution_ab.args[0][1].as_numer_denom()[0]
            c0 = solution_cd.args[0][0].as_numer_denom()[0]
            d0 = solution_cd.args[0][1].as_numer_denom()[0]
            a1 = pow(solution_ab.args[0][0].as_numer_denom()[1],-1,n)
            b1 = pow(solution_ab.args[0][1].as_numer_denom()[1],-1,n)
            c1 = pow(solution_cd.args[0][0].as_numer_denom()[1],-1,n)
            d1 = pow(solution_cd.args[0][1].as_numer_denom()[1],-1,n)

            a,b,c,d = [
                int(a0*a1)%26, int(b0*b1)%26,
                int(c0*c1)%26, int(d0*d1)%26
            ]

            key = "".join(self.alphabet[i] for i in [a,b,c,d])
            try:
                assert HillCipher(key).encodeSequence(self.string).lower()==self.encoded_string
                return key
            except:
                pass
        raise Exception("unable to find the key")

### Hill encoding Key Deduction

In [14]:
#Group 2
hd = HillKeyDeduction("politechnika".lower(),"CVWZATFEEZOA".lower())
key = hd.deduceKeyWithGaussMethod()
key

'ehni'